In [1]:
# Requires pymongo 3.6.0+
from pymongo import MongoClient
from bson.code import Code

client = MongoClient("mongodb://bigdata-mongodb-04.virtual.uniandes.edu.co:8087/", retryWrites=False)
database = client["Grupo03"]
collection = database["ARG_dataset"]

In [8]:
map = Code("function () {"
            'var regex = /[!"#$%&' + "'()*+,-./:;<=>?[\]^_`{|}~]/g;"
            "var palabras = this.reply_or_quote.split(' ');"       
            "palabras.forEach(function(z) {"
            "z = z.replace(regex, '');"
            "z = z.replace(/\\n/g,"");"
            "emit(z.toLowerCase(), 1);"
            "});"
            "}")

In [9]:
reduce = Code("function (key, values) {"
               "  var total = 0;"
               "  for (var i = 0; i < values.length; i++) {"
               "    total += values[i];"
               "  }"
               "  return total;"
               "}")

## Contador de palabras / test

In [10]:
result = database.COL_dataset.map_reduce(map, reduce, "emociones")
#for doc in result.find():
#    print(doc)

In [11]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('spanish')) 

collection = database["emociones"]
query = {}
query["_id"] = {
    u"$exists": True
}

projection = {}
projection["_id"] = 1.0
stopwords = {'q', 'si', 'usted', 'tan', 'solo', 'ser', 'bien', 'así', 'mas', 'va', 'van', 'señor', 'hace', 'hacer', 
             'siempre', 'gracias', 'favor', 'puede'}
stop_words.update(stopwords)

cursor = collection.find(query, projection = projection)
try:
    for doc in cursor:
        if (doc['_id'] in stop_words or (doc['_id'][0:1] == '@')):
            collection.delete_one({'_id': doc['_id']})
finally:
    client.close()

In [12]:
client.close()

## N-gramas

In [6]:
import pandas as pd
collection = database["ARG_dataset"]

# Created with Studio 3T, the IDE for MongoDB - https://studio3t.com/

query = {}
projection = {}
projection["reply_or_quote"] = 1.0

data = []
cursor = collection.find(query, projection = projection)
try:
    for doc in cursor:
        data.append([doc['_id'], doc['reply_or_quote']])
finally:
    client.close()


In [7]:
df = pd.DataFrame(data,columns=['_id', 'text'])
df.head()

,_id,text
0,5e91f3c26de7b8ddc68afa72,@alferdez Por favor Alberto seguí con la cuare...
1,5e91f3cf6de7b8ddc68afa73,@alferdez Grandes señor Presidente no le voté?...
2,5e91f3cf6de7b8ddc68afa74,"@alferdez Gracias a usted, presidente"
3,5e91f3cf6de7b8ddc68afa75,"@alferdez Si no ayudas a las pymes,hasta ahora..."
4,5e91f3cf6de7b8ddc68afa76,@alferdez Gracias ❤\n💕Gracias\nGracias ✌❤


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_n_words(corpus, n=1,k=1):
    vec = CountVectorizer(ngram_range=(k,k),stop_words = stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [9]:
good_pos = ['nc00000', 'nc0n000', 'nc0p000', 'nc0s000', 'np00000']

def ExtractInteresting(sentence, good):
    words = nltk.word_tokenize(sentence)
    interesting = [k for k,v in nltk.pos_tag(words) if v in good]
    print(interesting)
    return(interesting)

In [11]:
import nltk
from nltk.tag import StanfordPOSTagger
import re

tagger="stanford-postagger-full-2018-10-16/models/spanish.tagger"
jar="stanford-postagger-full-2018-10-16/stanford-postagger.jar"

spanish_postagger = StanfordPOSTagger(tagger,jar, encoding='utf8')


sentences = ['@ClaudiaLopez @infopresidencia @Bogota La verdad esa ayuda es una  mentira soy madre cabeza de familia no aparezco en el Sisben y aún Haci no he resubido ningún ayuda se supo e que las personas que no aparecen en el Sisben las localizavan de otra forma @ClaudiaLopez me urge una ayuda','@ClaudiaLopez @infopresidencia @Bogota no se señora alcaldesa si podamos contar con si apoyo ya q debido a esta emergencia mundial no he podido trabajar 😔agradeceria nos tenga en cuenta mi puntaje en el sisben es 18.47 y mi nivel es 1 quedo atenta a algún recibido sra claudia bendiciones y q mi Dios la proteja']

nouns = []
for index, row in df.iterrows():
    words = row['text'].split()
    tagged_words = spanish_postagger.tag(words)
    palabra_anterior = ''
    for (word, tag) in tagged_words:
        if tag in good_pos:
            if not ((word[0:1] == '@') or (word[0:4] == 'http')):
                word = re.sub(r'[^\w\s]','',word)
                #print(word+' '+tag)
                if palabra_anterior != '' and word != '':
                    pareja = palabra_anterior + ' ' + word
                    nouns.append(pareja.lower())
                palabra_anterior = word
#    del nouns[0]
#print(nouns)            

In [12]:
dfnouns = pd.DataFrame(nouns, columns=['text'])
collection_dataset = database["ARG_temas"]

for indice_fila, fila in dfnouns.iterrows():
    tweet = {'id': fila.text, 'frecuencia': ''}
    collection_dataset.insert_one(tweet)

In [ ]:
common_words = get_top_n_words(dfnouns['text'], 20,2)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])

## MapReduce (temas frecuentes)

In [5]:
collection = database["ARG_temas"]

In [25]:
map = Code("function () {"
            "var palabras = this.id;"            
            "emit(palabras, 1);"            
            "}")

In [26]:
reduce = Code("function (key, values) {"
               "  var total = 0;"
               "  for (var i = 0; i < values.length; i++) {"
               "    total += values[i];"
               "  }"
               "  return total;"
               "}")

In [27]:
result = database.ARG_temas.map_reduce(map, reduce, "ARG_temas_cuenta")